In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, Lasso

# 변수 앞에 붙는 계수를 압축해서 경향을 직관적으로 파악할 수 있게 데이터를 처리한다.

# 1. 데이터 준비 및 분할
df = pd.read_csv('https://bit.ly/perch_csv_data')
perch_full = df.to_numpy()
perch_weight = np.array(
    [5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0,
     110.0, 115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0,
     130.0, 150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0,
     197.0, 218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0,
     514.0, 556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0,
     820.0, 850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0,
     1000.0, 1000.0]
)

train_input, test_input, train_target, test_target = train_test_split(perch_full, perch_weight, random_state=42)

In [ ]:
# 2. 특성 공학 및 스케일링
poly = PolynomialFeatures(degree=5, include_bias=False)
poly.fit(train_input)
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)

ss = StandardScaler()
ss.fit(train_poly)
train_scaled = ss.transform(train_poly)
test_scaled = ss.transform(test_poly)

print("모델의 계수 변화를 확인하기 위한 준비 완료.")
print(f"변환된 특성 개수: {train_scaled.shape[1]}개")
print("-" * 50)

In [ ]:
# 릿지 규제 (alpha 값에 따른 계수 변화)
print("### 릿지 모델의 계수 변화 ###")
ridge_coefs = []
alpha_list = [0.01, 0.1, 1, 10, 100]
for alpha in alpha_list:
    ridge = Ridge(alpha=alpha)
    ridge.fit(train_scaled, train_target)
    ridge_coefs.append(ridge.coef_)
    print(f"alpha={alpha:<5} | 계수 크기(L2 Norm): {np.linalg.norm(ridge.coef_):.2f}")

In [ ]:
# 릿지 계수 변화 시각화
plt.figure(figsize=(10, 6))
# np.array(ridge_coefs).T -> ridge_coefs로 수정
plt.plot(np.log10(alpha_list), ridge_coefs) 
plt.xlabel('log10(alpha)')
plt.ylabel('coefficient size')
plt.title('Ridge Coefficients vs. alpha')
plt.show()

print("-" * 50)


In [ ]:
# 라쏘 규제 (alpha 값에 따른 계수 변화)
print("### 라쏘 모델의 계수 변화 ###")
lasso_coefs = []
for alpha in alpha_list:
    lasso = Lasso(alpha=alpha, max_iter=100000)
    lasso.fit(train_scaled, train_target)
    lasso_coefs.append(lasso.coef_)
    zero_coefs = np.sum(lasso.coef_ == 0)
    print(f"alpha={alpha:<5} | 0이 된 계수 개수: {zero_coefs}개 | 계수 크기(L1 Norm): {np.linalg.norm(lasso.coef_, 1):.2f}")

In [ ]:
# 라쏘 계수 변화 시각화
plt.figure(figsize=(10, 6))
# np.array(lasso_coefs).T -> lasso_coefs로 수정
plt.plot(np.log10(alpha_list), lasso_coefs) 
plt.xlabel('log10(alpha)')
plt.ylabel('coefficient size')
plt.title('Lasso Coefficients vs. alpha')
plt.show()